In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import struct
from scipy.signal import butter, filtfilt, firls, iircomb, resample_poly, decimate, envelope, resample, find_peaks
from scipy.io.wavfile import read as wavread

In [ ]:
path = 'mediciones_simultaneas/tase-david291225-20051201003131/bmecg'

with open(path, 'rb') as f:
    raw_data = f.read()

# 1. Parse the Header
# The file starts with "BMECG1" followed by a JSON object
header_start = 6  # Skip "BMECG1"
header_end = raw_data.find(b'}}') + 2
header_json = raw_data[header_start:header_end].decode('utf-8')
header = json.loads(header_json)

print("Header parsed successfully!")
print(f"Sampling Rate: {header['ecg']['samplingRate']} Hz")
print(f"Channels: {header['ecg']['channels']}")

# 2. Find Data Start
# Skip null bytes padding after the JSON header
data_start = header_end
while data_start < len(raw_data) and raw_data[data_start] == 0:
    data_start += 1

print(f"Data starts at byte: {data_start}")

# 3. Read Signal Data
# 'short' format usually means 16-bit signed integer (int16)
num_channels = len(header['ecg']['channels'])
signal_data = np.frombuffer(raw_data, dtype=np.int16, offset=data_start)

# Reshape data: (Samples, Channels)
# We truncate any extra bytes that don't fit perfectly into the shape
num_samples = len(signal_data) // num_channels
signal_data = signal_data[:num_samples * num_channels]
ecg_signal = signal_data.reshape(num_samples, num_channels)
print(f"ECG Signal Shape: {ecg_signal.shape}")
ecg_signal = ecg_signal[:, :12]  # Keep only first 12 channels if more are present

# bandpass filter between 0.5 Hz and 150 Hz for ECG signal channel 2
lowcut = 0.5
highcut = 60.0
fs = header['ecg']['samplingRate']
order = 3
b, a = butter(order, [lowcut, highcut], btype='bandpass', fs=fs)
b2, a2 = butter(order+1, [lowcut, highcut], btype='bandpass', fs=fs)

# comb filter at w0=50 Hz to remove powerline noise
w0 = 50.0  # Frequency to be removed from signal (Hz)
quality_factor = 30.0  # Quality factor
b_notch, a_notch = iircomb(w0, quality_factor, ftype='notch', fs=fs)
ecg_signal = np.array(ecg_signal)
ecg_signal = ecg_signal.astype(np.float32)

ecg_signal = filtfilt(b, a, ecg_signal, axis=0)
ecg_signal = filtfilt(b_notch, a_notch, ecg_signal, axis=0)
ecg_signal = filtfilt(b2, a2, ecg_signal, axis=0)
# increase sampling rate to 1000 Hz
ecg_resampled = resample_poly(ecg_signal[:, 1], 2, 1)
fs = fs * 2
b3, a3 = butter(order+1, [lowcut, highcut], btype='bandpass', fs=fs)
ecg_resampled = filtfilt(b3, a3, ecg_resampled)
ecg_normalized = ecg_resampled / np.max(np.abs(ecg_resampled))

fs, pcg = wavread('mediciones_simultaneas/tase-david291225-20051201003131/david291225_filtered_kalman.wav')
pcg = resample_poly(pcg, 10000, 10001)
envol = envelope(pcg, residual=None)
envol_normalized = envol / np.max(np.abs(envol))

n = 74  # calculated shift
ecg_normalized_shifted = ecg_normalized[n:]

x = np.arange(len(envol_normalized))
x_c = x * (299019/299410)

envol_resampled = np.interp(x, x_c, envol_normalized, right=np.nan)
pcg_resampled = np.interp(x, x_c, pcg, right=np.nan)
envol_resampled = envol_resampled[~np.isnan(envol_resampled)]
pcg_resampled = pcg_resampled[~np.isnan(pcg_resampled)]
plt.figure(figsize=(30, 10))
plt.plot(x[:len(envol_resampled)], ecg_normalized_shifted[:len(envol_resampled)], label='ECG Normalized Shifted')
plt.plot(x[:len(envol_resampled)], envol_resampled, label='PCG Envelope Normalized Shifted')
plt.xlim(290000, len(envol_resampled))
plt.show()

Header parsed successfully!
Sampling Rate: 500 Hz
Channels: ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'CM5', 'CR5']
Data starts at byte: 4096
ECG Signal Shape: (150215, 14)
